## **2 кейс**

**Выгрузка активности с ItResume**

**Важно**

Перед началом решения выполните следующую ячейку, чтобы загрузить необходимый для работы файл.

In [ ]:
!wget https://gist.github.com/Vs8th/a7a7f00e6cdef1b3fe87e4d61ca56e5f/raw/codesubmit.csv

--2025-10-20 15:39:37--  https://gist.github.com/Vs8th/a7a7f00e6cdef1b3fe87e4d61ca56e5f/raw/codesubmit.csv
Resolving gist.github.com (gist.github.com)... 140.82.116.4
Connecting to gist.github.com (gist.github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/a7a7f00e6cdef1b3fe87e4d61ca56e5f/raw/codesubmit.csv [following]
--2025-10-20 15:39:37--  https://gist.githubusercontent.com/Vs8th/a7a7f00e6cdef1b3fe87e4d61ca56e5f/raw/codesubmit.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 215378 (210K) [text/plain]
Saving to: ‘codesubmit.csv’

codesubmit.csv      100%[===================>] 210.33K  --.-KB/s    in 0.02s   

2025-10-20 15:39:37 (13.0 MB/s) - ‘co

Чтобы посмотреть как он выглядит выполните следующую ячейку.

In [ ]:
import pandas as pd

df = pd.read_csv('codesubmit.csv', sep = ';')
df

,created_at,user_id,problem_id,is_correct,type
0,2023-04-30 13:47:14.344471,7,870,1.0,submit
1,2023-04-30 13:46:15.949925,7,870,0.0,submit
2,2023-04-30 16:13:26.005286,173,21,1.0,submit
3,2023-04-30 16:13:06.739782,173,21,NaN,run
4,2023-04-30 15:52:00.195532,173,25,1.0,submit
...,...,...,...,...,...
4994,2023-04-30 21:52:00.269123,13493,435,NaN,run
4995,2023-04-30 21:51:01.094234,13493,435,1.0,submit
4996,2023-04-30 21:50:52.059690,13493,435,NaN,run
4997,2023-04-30 21:42:24.323689,13493,1086,NaN,run


### **Решения**

#### **Задача 1**

Ваша задача - выяснить сколько в среднем тратится времени на решение задачи.

**Примечание**: для правильного подсчета - рассчитайте сначала среднее время решения по каждой задаче в отдельности, и только затем находите общее среднее время решения задач.

Результат - число типа `float`, округлите до 2 знаков после запятой и запишите в переменную `res`.


**Решение**

Напишите свое решение ниже

In [ ]:
import csv
from datetime import datetime
from datetime import timedelta
from collections import defaultdict

with open('codesubmit.csv', 'r') as file:
  reader = csv.DictReader (file, delimiter=';')

  problem_by_user = defaultdict (list)
#переделываем формат даты
  for row in reader:
    key = (row['user_id'], row['problem_id'])
    problem_by_user[key].append((
        datetime.strptime(row['created_at'], '%Y-%m-%d %H:%M:%S.%f'),
        row['is_correct'],
        row['type']))

  first_atempsts = {}
  first_right_solutions = {}
  attempts_right_solutions = {}
#сортируем все по возрастанию даты
#отбираем певые неправильные попытки.
#Если неправильные поптки есть, то отбдираем и первое правильное решение
  for key, attempts in problem_by_user.items():
    attempts.sort (key = lambda x: x[0])
    if len (attempts)>1:
      for attempt in attempts:
        if attempt[1] == '' or int(attempt[1]) == 0:
          if key not in first_atempsts:
            first_atempsts[key] = attempt
        elif int(attempt[1]) == 1:
          first_right_solutions[key] = attempt
          break

  #отбираем те (юзерб, задача) где есть и правильные и неправильные попытки

  attempts_right_solutions = {}
  for key in problem_by_user.keys():
    first_atempt=first_atempsts.get(key)
    first_right_solution=first_right_solutions.get(key)
    if first_atempt is not None and first_right_solution is not None:
      attempts_right_solutions[key] = (first_atempt, first_right_solution)


  #считаем время выполнения каждой задачи пользователями
  time_task=defaultdict(list)
  for key, problem in attempts_right_solutions.items():
    delta_seconds= (problem[1][0]-problem[0][0]).total_seconds()
    time_task[key[1]].append(delta_seconds)

  #считаем среднее время на каждую задачу
  avg_time_task = {key: sum(delta)/len(delta) for key, delta in time_task.items()}

  #считаем среднее время по всем задачам
  avg_time_1_task = sum(avg_time_task.values())/len(avg_time_task)
  res = round(avg_time_1_task,2)
  print (res)


611.86


✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [ ]:
try:
    assert res == 611.86
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


#### **Задача 2**

Ваша задача - выяснить сколько часов в среднем проводит юзер в день на платформе. Перерывы в активности за день - не учитываем.

Результат - число типа `float`, округлите до 2 знаков после запятой и запишите в переменную `res2`.

**Решение**

Напишите свое решение ниже

In [ ]:
import csv
from datetime import datetime
from collections import defaultdict

with open ('codesubmit.csv', 'r') as file:
  reader=csv.DictReader(file, delimiter = ';')

  activity_by_user = defaultdict(list)
  for row in reader:
    key = row['user_id']
    activity_by_user[key].append(datetime.strptime(row['created_at'],'%Y-%m-%d %H:%M:%S.%f'))

  user_sessions=defaultdict(list)
  for user, activities in activity_by_user.items():
    for activity in activities:
      day=activity.date()
      user_sessions[(user, day)].append(activity)

  total_hours_by_day = {}
  for (user,day), acivity in user_sessions.items():
    acivity.sort()
    delta=(acivity[-1]-acivity[0]).total_seconds()
    total_hours_by_day[(user,day)]=delta

  avg_time_sec=sum(total_hours_by_day.values())/len(total_hours_by_day)
  avg_time_hours=round(avg_time_sec/3600,2)

res2 = avg_time_hours

1.7


✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [ ]:
try:
    assert res2 == 1.7
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


#### **Задача 3**

Теперь давайте посмотрим на активные сеансы. Выясните, сколько задач в среднем решается за один активный сеанс.

**Активный сеанс** - период, когда между любой активностью пользователя разница менее или равна часу, не более

**Важно**: в расчет берем не только успешные попытки решений (`is_correct=1`), а и неуспешные тоже (`is_correct=0`), и тип `run` в том числе.

Результат - число типа `float`, округлите до 2 знаков после запятой и запишите в переменную `res3`.

**Решение**

Напишите свое решение ниже

In [ ]:
import csv
from datetime import datetime, timedelta
from collections import defaultdict

with open ('codesubmit.csv', 'r') as file:
  reader = csv.DictReader(file, delimiter=';')

  activity_by_user = defaultdict(list)
  for row in reader:
    activity_by_user[row['user_id']].append((datetime.strptime(row['created_at'],'%Y-%m-%d %H:%M:%S.%f'), row['problem_id']))

  #сортируем по времени в разрезе юзер_задача
  user_sessions=defaultdict(list)
  for user_id, activities_problem_id in activity_by_user.items():
    activities_problem_id.sort()
    session=1
  #разделяем на сеансы
    user_sessions[(user_id, session)].append(activities_problem_id[0])  # добавляем первую активность в текущий сеанс

    for i in range (1, len(activities_problem_id)):
      current_activity=activities_problem_id[i]
      previous_activity=activities_problem_id[i-1]
      if current_activity[0]-previous_activity[0]<=timedelta(hours=1):
        user_sessions[(user_id, session)].append(current_activity)
      else:
        session+=1
        user_sessions[(user_id, session)].append(current_activity)

  #ключ-пользователь: значение - уникальнаяя пара задача-номер сеанса
  user_task_session = defaultdict(set)
  for key, values in user_sessions.items():
    for value in values:
      user_task_session[key[0]].add((value[1], key[1]))

  #ключ-(пользователь, номер сессии): значение - количество уникльных задач
  cnt_dict=defaultdict(int)
  for user, task_session in user_task_session.items():
    for task in task_session:
      cnt_dict[(user, task[1])]+=1

  res3 = round(sum(cnt_dict.values())/len(cnt_dict),2)

  print(res3)


3.14


✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [ ]:
try:
    assert res3 == 3.14
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


#### **Задача 4**

И финальная - найдите самый "популярный" час дня на нашей платформе.

Популярность определяем максимальным количеством уникальных пользователей, совершающих какую-либо активность в этот период

Результат в числовом формате запишите в переменную `res4`.

Например, самым популярным часом стал период с 22 до 23, тогда в переменной `res4` должно лежать **22**. Обозначающее начало этого периода.

**Решение**

Напишите свое решение ниже

In [ ]:
import csv
from datetime import datetime, timedelta
from collections import defaultdict

with open ('codesubmit.csv', 'r') as file:
  reader = csv.DictReader(file, delimiter=';')

  #ключ-юзер, значения - уникальные часы
  activity_by_user = defaultdict(set)
  for row in reader:
    key = row['user_id']
    activity_by_user[key].add(datetime.strptime(row['created_at'],'%Y-%m-%d %H:%M:%S.%f').hour)

  hour_users = defaultdict(set)
  for user, hours in activity_by_user.items():
    for hour in hours:
      hour_users[hour].add(user)

res4=max(hour_users.keys(), key=lambda key: len(hour_users[key]))

print(res4)

16


✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [ ]:
try:
    assert res4 == 16
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!
